In [1]:
import pandas as pd
import pydicom
import numpy as np
# Set the path to the dataset folder
dataset_path = "./cbis-ddsm-dataset/"

# Load the mass case description file
mass_case_description = pd.read_csv(dataset_path + "mass_case_description_train_set.csv")
metadata = pd.read_csv(dataset_path + "metadata.csv")

In [2]:
mass_case_description["cropped image file path"] = dataset_path + "CBIS-DDSM/" + mass_case_description["cropped image file path"]
mass_case_description["ROI mask file path"] = dataset_path + "CBIS-DDSM/" + mass_case_description["ROI mask file path"]
# Define the regex pattern
pattern = r'(.*/)[^/]*$'

# Extract the directory paths
mass_case_description["directory_path_cropped_image"] = mass_case_description["cropped image file path"].str.extract(pattern)
mass_case_description["directory_path_roi_image"] = mass_case_description["ROI mask file path"].str.extract(pattern)

from glob import glob
from pathlib import Path

# Define a function to find the correct DICOM file in the directory
def find_dicom_file(directory):
    search_path = Path(directory) / "*.dcm"
    dicom_files = glob(str(search_path))

    if len(dicom_files) > 0:
        return dicom_files[0]
    else:
        return None

# Find the actual corresponding file paths
mass_case_description["actual_file_path_cropped"] = mass_case_description["directory_path_cropped_image"].apply(find_dicom_file)
mass_case_description["actual_file_path_roi"] = mass_case_description["directory_path_roi_image"].apply(find_dicom_file)

print(mass_case_description["actual_file_path_cropped"][0], mass_case_description["actual_file_path_roi"][0])

mass_case_description.to_csv("./cbis-ddsm-dataset/updated_mass_case_description_train_set.csv")

cbis-ddsm-dataset/CBIS-DDSM/Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.9590.100.1.2.108268213011361124203859148071588939106/1.3.6.1.4.1.9590.100.1.2.296736403313792599626368780122205399650/1-2.dcm cbis-ddsm-dataset/CBIS-DDSM/Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.9590.100.1.2.108268213011361124203859148071588939106/1.3.6.1.4.1.9590.100.1.2.296736403313792599626368780122205399650/1-2.dcm


In [1]:
import numpy as np
import pandas as pd
import dicomsdl
import os
import re
from concurrent.futures import ThreadPoolExecutor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
# from skimage.transform import resize
import cv2
# Load the CSV file
dataset_path = "./cbis-ddsm-dataset/"
mass_case_data = pd.read_csv(dataset_path + "updated_mass_case_description_train_set.csv")

# Define a function to extract image data from a DICOM file
def extract_image_data(file_path):
    dicom_data = dicomsdl.open(file_path)
    image_data = dicom_data.pixelData()
    image_data = cv2.resize(image_data, (512, 512))
    image_data = np.expand_dims(image_data, axis=-1)
    return image_data

# Define a function to extract data from the DataFrame in parallel using multiple threads
def extract_data_parallel(mass_case_data, n_threads=8):
    image_paths = mass_case_data["actual_file_path_cropped"]

    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        images = list(executor.map(extract_image_data, image_paths))

    breast_density = mass_case_data["breast_density"].values
    subtlety = mass_case_data["subtlety"].values
    side = mass_case_data["left or right breast"].values
    view = mass_case_data["image view"].values

    pathology = mass_case_data["pathology"].values
    assessment = mass_case_data["assessment"].values
    shape = mass_case_data["mass shape"].values
    margin = mass_case_data["mass margins"].values

    return images, breast_density, subtlety, side, view, pathology, assessment, shape, margin

# Extract the data using multithreading
images, breast_density, subtlety, side, view, pathology, assessment, shape, margin = extract_data_parallel(mass_case_data)

In [2]:
tf.__version__

'2.6.0'

In [3]:
#Setting gpu for limit memory
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    #Restrict Tensorflow to only allocate 6gb of memory on the first GPU
   try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0],
       [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7392)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
   except RuntimeError as e:
       #virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-04-26 07:58:09.629670: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 07:58:09.643612: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 07:58:09.643749: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 07:58:09.657316: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
breast_density_encoder = LabelEncoder()
subtlety_encoder = LabelEncoder()
side_encoder = LabelEncoder()
view_encoder = LabelEncoder()
pathology_encoder = LabelEncoder()
assessment_encoder = LabelEncoder()
shape_encoder = LabelEncoder()
margin_encoder = LabelEncoder()

breast_density_encoded = breast_density_encoder.fit_transform(breast_density)
subtlety_encoded = subtlety_encoder.fit_transform(subtlety)
side_encoded = side_encoder.fit_transform(side)
view_encoded = view_encoder.fit_transform(view)
pathology_encoded = pathology_encoder.fit_transform(pathology)
assessment_encoded = assessment_encoder.fit_transform(assessment)
shape_encoded = shape_encoder.fit_transform(shape)
margin_encoded = margin_encoder.fit_transform(margin)


pathology_one_hot = to_categorical(pathology_encoded)
breast_density_one_hot = to_categorical(breast_density_encoded)
subtlety_one_hot = to_categorical(subtlety_encoded)
side_one_hot = to_categorical(side_encoded)
view_one_hot = to_categorical(view_encoded)
assessment_one_hot = to_categorical(assessment_encoded)
shape_one_hot = to_categorical(shape_encoded)
margin_one_hot = to_categorical(margin_encoded)

# pathology_one_hot = tf.expand_dims(pathology_one_hot, axis=-1)
images = np.array(images, dtype=np.float32) / 255.
print(pathology_one_hot.shape)

(images_train, images_val,
 side_train, side_val,
 view_train, view_val,
 breast_density_train, breast_density_val,
 subtlety_train, subtlety_val,
 pathology_train, pathology_val,
 assessment_train, assessment_val,
 shape_train, shape_val,
 margin_train, margin_val) = train_test_split(images, side_one_hot, view_one_hot, breast_density_one_hot, subtlety_one_hot,
                                              pathology_one_hot, assessment_one_hot, shape_one_hot, margin_one_hot,
                                              test_size=0.2, random_state=42)

print(pathology_train.shape, assessment_train.shape, shape_train.shape, margin_train.shape)
train_dataset = tf.data.Dataset.from_tensor_slices(((images_train, side_train, view_train, breast_density_train, subtlety_train),
            (pathology_train, assessment_train, shape_train, margin_train))).shuffle(buffer_size=1000).batch(32).cache().prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices(((images_val, side_val, view_val, breast_density_val, subtlety_val),
            (pathology_val, assessment_val, shape_val, margin_val))).batch(32).cache().prefetch(tf.data.AUTOTUNE)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060 Ti, compute capability 8.6


2023-04-26 07:58:13.232570: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


(1318, 3)
(1054, 3) (1054, 6) (1054, 19) (1054, 16)


2023-04-26 07:58:13.751798: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1105199104 exceeds 10% of free system memory.
2023-04-26 07:58:14.233481: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1105199104 exceeds 10% of free system memory.


In [4]:
len(train_dataset)

33

In [5]:
from keras.callbacks import TensorBoard, ModelCheckpoint

tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

checkpoint_callback = ModelCheckpoint(
    filepath='./checkpoints/model_checkpoint_{epoch:02d}',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

callbacks = [tensorboard_callback, checkpoint_callback]

2023-04-26 07:58:35.813706: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2023-04-26 07:58:35.813731: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2023-04-26 07:58:35.813770: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs
2023-04-26 07:58:35.814786: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-26 07:58:36.029063: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2023-04-26 07:58:36.029206: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed


In [6]:
import tensorflow as tf
from tensorflow.keras import layers, Model
# Define the custom model
class MultiInputMultiOutputModel(tf.keras.Model):
    def __init__(self):
        super(MultiInputMultiOutputModel, self).__init__()
        self.conv1 = layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 1))
        self.pool1 = layers.MaxPooling2D((2, 2))
        self.conv2 = layers.Conv2D(64, (3, 3), activation='relu')
        self.pool2 = layers.MaxPooling2D((2, 2))
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dropout = layers.Dropout(0.5)
        self.dense2 = layers.Dense(32, activation='relu')
        self.dense3 = layers.Dense(128, activation="relu")
        self.pathology_output = layers.Dense(3, activation='softmax', dtype='float32', name='pathology_output')
        self.assessment_output = layers.Dense(6, activation='softmax', dtype='float32', name='assessment_output')
        self.shape_output = layers.Dense(19, activation='softmax', dtype='float32', name='shape_output')
        self.margin_output = layers.Dense(16, activation='softmax', dtype='float32', name='margin_output')

    def call(self, inputs):
        images, side, view, breast_density, subtlety = inputs
        x = self.conv1(images)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        
        # side = tf.cast(side, tf.float32)
        # view = tf.cast(view, tf.float32)
        # breast_density = tf.cast(breast_density, tf.float32)
        # subtlety = tf.cast(subtlety, tf.float32)

        x = tf.concat([x, side, view, breast_density, subtlety], axis=-1)
        x = self.dense2(x)
        x = self.dense1(x)
        x = self.dense3(x)

        pathology_output = self.pathology_output(x)
        assessment_output = self.assessment_output(x)
        shape_output = self.shape_output(x)
        margin_output = self.margin_output(x)

        return [pathology_output , assessment_output, shape_output, margin_output]

# Instantiate the custom model
model = MultiInputMultiOutputModel()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [7]:
history = model.fit(train_dataset, batch_size=8, validation_batch_size=8, validation_data=val_dataset, epochs=20, steps_per_epoch=len(train_dataset),
                    validation_steps=len(val_dataset), callbacks=callbacks)

2023-04-26 07:59:05.924643: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1105199104 exceeds 10% of free system memory.
2023-04-26 07:59:06.452294: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1105199104 exceeds 10% of free system memory.


Epoch 1/20


2023-04-26 07:59:07.690211: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2023-04-26 07:59:08.325381: E tensorflow/stream_executor/cuda/cuda_dnn.cc:374] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2023-04-26 07:59:08.325609: E tensorflow/stream_executor/cuda/cuda_dnn.cc:382] Possibly insufficient driver version: 525.105.17
2023-04-26 07:59:08.326166: E tensorflow/stream_executor/cuda/cuda_dnn.cc:374] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node multi_input_multi_output_model/conv2d/Conv2D (defined at tmp/ipykernel_1501/1238435592.py:23) ]] [Op:__inference_train_function_2640]

Errors may have originated from an input operation.
Input Source operations connected to node multi_input_multi_output_model/conv2d/Conv2D:
 multi_input_multi_output_model/Cast (defined at tmp/ipykernel_1501/694485789.py:1)

Function call stack:
train_function


In [8]:
model.summary()

Model: "multi_input_multi_output_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  320       
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           multiple                  18496     
                                                                 
 max_pooling2d_1 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multipl

In [9]:
checkpoint_callback_1 = ModelCheckpoint(
    filepath='./checkpoints/model_1_checkpoint_{epoch:02d}.hdf5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

callbacks=[tensorboard_callback, checkpoint_callback_1]

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
tf.keras.backend.clear_session()
# Define the custom model
class MultiInputMultiOutputModel_V2B0(tf.keras.Model):
    def __init__(self):
        super(MultiInputMultiOutputModel_V2B0, self).__init__()
        self.input_layer = layers.InputLayer(input_shape=((512,512,1)))
        self.eff_layer = tf.keras.applications.efficientnet_v2.EfficientNetV2B3(include_top=False, trainable=False)     
        
        self.global_avg = layers.GlobalAveragePooling2D()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dropout = layers.Dropout(0.5)
        self.dense2 = layers.Dense(32, activation='relu')
        
        self.pathology_output = layers.Dense(3, activation='softmax', name='pathology_output')
        self.assessment_output = layers.Dense(6, activation='softmax', name='assessment_output')
        self.shape_output = layers.Dense(19, activation='softmax', name='shape_output')
        self.margin_output = layers.Dense(16, activation='softmax', name='margin_output')
        self.eff_layer.trainable = False
        

    def call(self, inputs):
        images, side, view, breast_density, subtlety = inputs
        x = self.input_layer(images)
        x = self.eff_layer(x)
        x = self.global_avg(x)
        
        # side = tf.cast(side, tf.float32)
        # view = tf.cast(view, tf.float32)
        # breast_density = tf.cast(breast_density, tf.float32)
        # subtlety = tf.cast(subtlety, tf.float32)

        x = tf.concat([x, side, view, breast_density, subtlety], axis=-1)
        # x = self.batchnormal(x)
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)

        pathology_output = self.pathology_output(x)
        assessment_output = self.assessment_output(x)
        shape_output = self.shape_output(x)
        margin_output = self.margin_output(x)

        return [pathology_output , assessment_output, shape_output, margin_output]

# Instantiate the custom model
model_1 = MultiInputMultiOutputModel_V2B0()

# Compile the model
model_1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
model_1_history = model_1.fit(train_dataset, batch_size=8, validation_batch_size=8, validation_data=val_dataset, epochs=20, callbacks=callbacks)

In [5]:


# # pathology_one_hot = tf.expand_dims(pathology_one_hot, axis=-1)
# images = np.array(images, dtype=np.float32) / 255.
# print(pathology_one_hot.shape)

# (images_train, images_val,
#  side_train, side_val,
#  view_train, view_val,
#  breast_density_train, breast_density_val,
#  subtlety_train, subtlety_val,
#  pathology_train, pathology_val,
#  assessment_train, assessment_val,
#  shape_train, shape_val,
#  margin_train, margin_val) = train_test_split(images, side_one_hot, view_one_hot, breast_density_one_hot, subtlety_one_hot,
#                                               pathology_one_hot, assessment_one_hot, shape_one_hot, margin_one_hot,
#                                               test_size=0.2, random_state=42)

# print(pathology_train.shape, assessment_train.shape, shape_train.shape, margin_train.shape)
# train_dataset = tf.data.Dataset.from_tensor_slices(((images_train, side_train, view_train, breast_density_train, subtlety_train),
#             (pathology_train, assessment_train, shape_train, margin_train))).shuffle(buffer_size=1000).batch(32).cache().prefetch(tf.data.AUTOTUNE)
# val_dataset = tf.data.Dataset.from_tensor_slices(((images_val, side_val, view_val, breast_density_val, subtlety_val),
#             (pathology_val, assessment_val, shape_val, margin_val))).batch(32).cache().prefetch(tf.data.AUTOTUNE)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060 Ti, compute capability 8.6


2023-04-19 22:11:41.071647: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


(1318, 3)
(1054, 3) (1054, 6) (1054, 19) (1054, 16)


In [1]:
import numpy as np
import pandas as pd
import dicomsdl
from concurrent.futures import ThreadPoolExecutor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import cv2
tf.keras.mixed_precision.set_global_policy('mixed_float16')
# Load the CSV file
dataset_path = "./cbis-ddsm-dataset/"
mass_case_data = pd.read_csv(dataset_path + "updated_mass_case_description_train_set.csv")

# Define a function to extract image data from a DICOM file
def extract_image_data(file_path):
    dicom_data = dicomsdl.open(file_path)
    image_data = dicom_data.pixelData()
    image_data = cv2.resize(image_data, (512, 512))
    image_data = np.stack([image_data] * 3, axis=-1)
    return image_data

# Define a function to extract data from the DataFrame in parallel using multiple threads
def extract_data_parallel(mass_case_data, n_threads=8):
    image_paths = mass_case_data["actual_file_path_cropped"]

    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        images = list(executor.map(extract_image_data, image_paths))

    breast_density = mass_case_data["breast_density"].values
    subtlety = mass_case_data["subtlety"].values
    side = mass_case_data["left or right breast"].values
    view = mass_case_data["image view"].values

    pathology = mass_case_data["pathology"].values
    assessment = mass_case_data["assessment"].values
    shape = mass_case_data["mass shape"].values
    margin = mass_case_data["mass margins"].values

    return images, breast_density, subtlety, side, view, pathology, assessment, shape, margin

# Extract the data using multithreading
images_1, breast_density, subtlety, side, view, pathology, assessment, shape, margin = extract_data_parallel(mass_case_data)
images_1 = np.array(images_1, dtype=np.float32) / 255.

from tensorflow.keras.utils import to_categorical
from tensorflow.keras import mixed_precision
breast_density_encoder = LabelEncoder()
subtlety_encoder = LabelEncoder()
side_encoder = LabelEncoder()
view_encoder = LabelEncoder()
pathology_encoder = LabelEncoder()
assessment_encoder = LabelEncoder()
shape_encoder = LabelEncoder()
margin_encoder = LabelEncoder()

breast_density_encoded = breast_density_encoder.fit_transform(breast_density)
subtlety_encoded = subtlety_encoder.fit_transform(subtlety)
side_encoded = side_encoder.fit_transform(side)
view_encoded = view_encoder.fit_transform(view)
pathology_encoded = pathology_encoder.fit_transform(pathology)
assessment_encoded = assessment_encoder.fit_transform(assessment)
shape_encoded = shape_encoder.fit_transform(shape)
margin_encoded = margin_encoder.fit_transform(margin)


pathology_one_hot = to_categorical(pathology_encoded)
breast_density_one_hot = to_categorical(breast_density_encoded)
subtlety_one_hot = to_categorical(subtlety_encoded)
side_one_hot = to_categorical(side_encoded)
view_one_hot = to_categorical(view_encoded)
assessment_one_hot = to_categorical(assessment_encoded)
shape_one_hot = to_categorical(shape_encoded)
margin_one_hot = to_categorical(margin_encoded)

print(images_1.shape)
(images_train, images_val,
 side_train, side_val,
 view_train, view_val,
 breast_density_train, breast_density_val,
 subtlety_train, subtlety_val,
 pathology_train, pathology_val,
 assessment_train, assessment_val,
 shape_train, shape_val,
 margin_train, margin_val) = train_test_split(images_1, side_one_hot, view_one_hot, breast_density_one_hot, subtlety_one_hot,
                                              pathology_one_hot, assessment_one_hot, shape_one_hot, margin_one_hot,
                                              test_size=0.2, random_state=42)

print(pathology_train.shape, assessment_train.shape, shape_train.shape, margin_train.shape)
train_dataset = tf.data.Dataset.from_tensor_slices(((images_train, side_train, view_train, breast_density_train, subtlety_train),
            (pathology_train, assessment_train, shape_train, margin_train))).shuffle(buffer_size=1000).batch(8).cache().prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices(((images_val, side_val, view_val, breast_density_val, subtlety_val),
            (pathology_val, assessment_val, shape_val, margin_val))).batch(8).cache().prefetch(tf.data.AUTOTUNE)

2023-04-25 14:16:43.216415: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-25 14:16:43.246415: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060 Ti, compute capability 8.6


2023-04-25 14:16:44.991075: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-25 14:16:45.000008: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-25 14:16:45.000331: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

(1318, 512, 512, 3)
(1054, 3) (1054, 6) (1054, 19) (1054, 16)


2023-04-25 14:17:00.177067: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-25 14:17:00.177254: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-25 14:17:00.177340: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
from vit_keras import vit
import tensorflow as tf

class ViTMultiInputMultiOutputModel(tf.keras.Model):
    def __init__(self):
        super(ViTMultiInputMultiOutputModel, self).__init__()
        
        # Define the ViT model
        self.vit_model = vit.vit_b16(
            image_size=512,
            activation='relu',
            pretrained=True,  # Load pre-trained weights
            include_top=False,  # Do not include the classification head
            pretrained_top=False,  # Do not load pre-trained weights for the classification head
            weights='imagenet21k+imagenet2012',
        )
        
        # Define the other input layers
        self.side = tf.keras.layers.InputLayer(input_shape=(1,))
        self.view = tf.keras.layers.InputLayer(input_shape=(1,))
        self.breast_density = tf.keras.layers.InputLayer(input_shape=(1,))
        self.subtlety = tf.keras.layers.InputLayer(input_shape=(1,))
        
        # Define the dense layers
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')

        # Define the output layers
        self.pathology_output = tf.keras.layers.Dense(3, activation='softmax', name="pathology_output")
        self.assessment_output = tf.keras.layers.Dense(6, activation='softmax', name="assessment_output")
        self.shape_output = tf.keras.layers.Dense(19, activation='softmax', name="shape_output")
        self.margin_output = tf.keras.layers.Dense(16, activation='softmax', name="margin_output")

    def call(self, inputs):
        image, side, view, breast_density, subtlety = inputs

        # Pass the image through the ViT model
        x = self.vit_model(image)

        # Flatten the output of the ViT model
        x = tf.keras.layers.Flatten()(x)

        # Concatenate the other inputs with the output of the ViT model
        x = tf.keras.layers.Concatenate()([x, side, view, breast_density, subtlety])

        # Pass the concatenated inputs through the dense layers
        x = self.dense1(x)
        x = self.dense2(x)

        # Generate the outputs
        pathology_output = self.pathology_output(x)
        assessment_output = self.assessment_output(x)
        shape_output = self.shape_output(x)
        margin_output = self.margin_output(x)

        return pathology_output, assessment_output, shape_output, margin_output


/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
# Create a model instance
model_2 = ViTMultiInputMultiOutputModel()

from keras.callbacks import TensorBoard, ModelCheckpoint
checkpoint_callback_2 = ModelCheckpoint(
    filepath='./checkpoints/model_2_checkpoint_{epoch:02d}.hdf5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

# Define input tensors
image_input = tf.keras.Input(shape=(256, 256, 3), name="image_input")
side_input = tf.keras.Input(shape=(1,), name="side_input")
view_input = tf.keras.Input(shape=(1,), name="view_input")
breast_density_input = tf.keras.Input(shape=(1,), name="breast_density_input")
subtlety_input = tf.keras.Input(shape=(1,), name="subtlety_input")

# Call the model with the input tensors
# Compile the model
model_3.compile(
    optimizer='adam',
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

# Fit the model
history_3 = model_3.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    batch_size=8,
    validation_batch_size=8,
    callbacks=[tensorboard_callback, checkpoint_callback_2]
)


/usr/local/lib/python3.8/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 32, 32
  warnings.warn(


Epoch 1/20


2023-04-25 14:17:06.333241: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype float and shape [1054,6]
	 [[{{node Placeholder/_4}}]]
2023-04-25 14:17:06.333630: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype float and shape [1054,4]
	 [[{{node Placeholder/_3}}]]
2023-04-25 14:17:26.517208: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-25 14:17:37.689697: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 192.38MiB (rounded to 201720064)requested by op vi_t_mul

ResourceExhaustedError: Graph execution error:

Detected at node 'vi_t_multi_input_multi_output_model/vit-b16/Transformer/encoderblock_1/MultiHeadDotProductAttention_1/Softmax' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 5